In [23]:
import os
import torch
import numpy as np
import glob
import albumentations
from PIL import Image
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim

In [4]:
DATA_PATH = '/home/jupyter/datascience/chest-xray-pneumonia/data/chest_xray'
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sabz2301","key":"68b5b084ae177eb3df511e61147b64fc"}'}

In [5]:
 ! mkdir ~/.kaggle 

In [8]:
 ! cp kaggle.json ~/.kaggle/
 ! ls ~/.kaggle/

kaggle.json


In [9]:
! kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

100% 2.29G/2.29G [00:42<00:00, 68.8MB/s]
100% 2.29G/2.29G [00:43<00:00, 57.3MB/s]


In [10]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:

! unzip chest-xray-pneumonia.zip 

In [28]:
class ClassificationDataset:
    def __init__(self,image_paths,targets,resize=None):
        self.image_paths=image_paths
        self.targets = targets
        self.resize = resize
        self.aug= albumentations.Compose([
            albumentations.Rotate(),
            albumentations.Normalize(always_apply=True,)
        ])
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self,item):
        path = self.image_paths[item]
        image = Image.open(path).convert("RGB")
        target = self.targets[item]
        if self.resize is not None:
            image =image.resize((self.resize[1],self.resize[0]),resample=Image.BILINEAR)
        
        image = np.array(image)
        augmented = self.aug(image=image)
        image = augmented["image"]
        image = np.transpose(image,(2,0,1)).astype(np.float32)
        image = torch.tensor(data=image,dtype=torch.float)
        return image,target

In [ ]:
! ls /content/chest_xray/train

NORMAL	PNEUMONIA


In [29]:
DATA_PATH
train_images = glob.glob("/content/chest_xray/train/**/**.jpeg")
train_targets = [(0 if (x.split("/")[-2]=='PNEUMONIA') else 1 ) for x in train_images]

valid_images = glob.glob("/content/chest_xray/val/**/**.jpeg")
valid_targets = [(0 if (x.split("/")[-2]=='PNEUMONIA') else 1 ) for x in valid_images]

train_ds = ClassificationDataset(train_images,train_targets,resize=(24,24))
valid_ds = ClassificationDataset(valid_images,valid_targets,resize=(24,24))

In [31]:
train_dl=data.DataLoader(dataset=train_ds,batch_size=64,shuffle=True)
next(iter(train_dl))

[tensor([[[[-1.3130, -1.2274, -1.4158,  ..., -1.3302, -1.2617, -1.3815],
           [-1.4500, -1.2445, -0.9363,  ..., -0.7822, -0.2684, -1.1418],
           [-0.8507, -0.2684,  0.1254,  ...,  0.0227,  0.0569, -0.2856],
           ...,
           [-1.3473, -0.4739,  1.1187,  ...,  0.2967, -1.1589, -1.2103],
           [-1.3473, -0.4226,  1.1358,  ...,  0.0912, -1.1418, -1.3302],
           [-1.3473, -0.3541,  1.1358,  ..., -0.0458, -1.2445, -1.3130]],
 
          [[-1.2129, -1.1253, -1.3179,  ..., -1.2304, -1.1604, -1.2829],
           [-1.3529, -1.1429, -0.8277,  ..., -0.6702, -0.1450, -1.0378],
           [-0.7402, -0.1450,  0.2577,  ...,  0.1527,  0.1877, -0.1625],
           ...,
           [-1.2479, -0.3550,  1.2731,  ...,  0.4328, -1.0553, -1.1078],
           [-1.2479, -0.3025,  1.2906,  ...,  0.2227, -1.0378, -1.2304],
           [-1.2479, -0.2325,  1.2906,  ...,  0.0826, -1.1429, -1.2129]],
 
          [[-0.9853, -0.8981, -1.0898,  ..., -1.0027, -0.9330, -1.0550],
           [-

In [32]:
model=models.resnet34(pretrained=True,progress=True,)
loss_function=F.cross_entropy
opt=optim.Adam(model.parameters(),weight_decay=0.01)

In [34]:
epochs=4
for epoch in range(epochs):
  for xb,yb in train_dl:
    pred = model(xb)
    loss = loss_function(pred,yb)
    loss.backward()
    opt.step()
    opt.zero_grad

loss for epoch 0 => 0.9168003797531128
loss for epoch 1 => 3.5230562686920166
loss for epoch 2 => 0.6696447730064392
loss for epoch 3 => 0.673809826374054


In [35]:
loss_function(model(xb),yb)

tensor(0.6738, grad_fn=<NllLossBackward>)